In [1]:
import math
from sklearn import neighbors, svm
import os
import os.path
import pickle
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder

In [2]:
train_dir = "ActorsDataset"

trainX = []
trainY = []
testX = []
testY = []

In [3]:
for class_dir in os.listdir(train_dir):

    nr = 0
    for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
        
        image = face_recognition.load_image_file(img_path)
        face_bounding_boxes = face_recognition.face_locations(image)

        if len(face_bounding_boxes) == 1:
            if nr < 30:
                testX.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                testY.append(class_dir)
            else:
                trainX.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                trainY.append(class_dir)
            nr = nr + 1

In [4]:
print(len(trainX))
print(len(trainY))
print(len(testX))
print(len(testY))

7654
7654
3000
3000


In [5]:
def trainKNN(model_save_path=None, n_neighbors=None, knn_algo='ball_tree'):
    
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(trainX))))

    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(trainX, trainY)

    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf

In [6]:
print("Training KNN classifier...")
knn_clf = trainKNN(model_save_path="trained_knn_model.clf", n_neighbors=2)
print("Training complete!")

Training KNN classifier...
Training complete!


In [7]:
def predictKNN(faces_encodings, distance_threshold=0.6):

    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][0][0] <= distance_threshold]
    
    if are_matches:
        return knn_clf.predict(faces_encodings)[0]
    else:
        return "unkown"

In [8]:
total = 0
nrCorecte = 0
for i in range(len(testX)):
    name = predictKNN([testX[i]])
    total = total + 1
    if name == testY[i]:
        nrCorecte = nrCorecte + 1
    else:
        print("Corect: " + testY[i] + "; Prezis: " + name)

print("Corecte KNN: " + str(nrCorecte))
print("Totale KNN: " + str(total))
print("Acc KNN: " + str(nrCorecte/total))

Corect: Madchen Amick; Prezis: Haley Bennett
Corect: Katherine Waterston; Prezis: Robin Wright
Corect: Dafne Keen; Prezis: Jessica Henwick
Corect: Tom Hardy; Prezis: Taron Egerton
Corect: Shailene Woodley; Prezis: Jennifer Lawrence
Corect: Laura Haddock; Prezis: Margot Robbie
Corect: Bryce Dallas Howard; Prezis: Halston Sage
Corect: Bryce Dallas Howard; Prezis: Emma Stone
Corect: Cara Delevingne; Prezis: Lily James
Corect: Halston Sage; Prezis: Connie Nielsen
Corecte KNN: 2990
Totale KNN: 3000
Acc KNN: 0.9966666666666667


In [ ]:
def trainSVM(model_save_path=None):

    svm_clf = svm.SVC(kernel='linear', C = 1.0)
    svm_clf.fit(trainX, trainY)
    
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(svm_clf, f)

    return svm_clf

In [ ]:
print("Training SVM classifier...")
svm_clf = trainSVM(model_save_path="trained_svm_model.clf")
print("Training complete!")

In [ ]:
def predictSVM(faces_encodings):
    return svm_clf.predict(faces_encodings)[0]

In [ ]:
total = 0
nrCorecte = 0
for i in range(len(testX)):
    name = predictSVM([testX[i]])
    total = total + 1
    if name == testY[i]:
        nrCorecte = nrCorecte + 1
    else:
        print("Corect: " + testY[i] + "; Prezis: " + name)

print("Corecte KNN: " + str(nrCorecte))
print("Totale KNN: " + str(total))
print("Acc KNN: " + str(nrCorecte/total))